# Notebook to analysze and summarize soildgrids data by latitude

In [ ]:
import geemap
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
ee.Initialize()

#### Soilgrids in GEE community catalog
https://gee-community-catalog.org/projects/isric/

In [ ]:
# Load the SoilGrids dataset for soil organic carbon stocks.
isric_ocs = ee.Image("projects/soilgrids-isric/ocs_mean")

# calculate attribute value multiplied with grid size, scale to km²
# gridcell = t/ha
pixel_area = ee.Image.pixelArea().multiply(1e-4) # grid size in ha
factor = 1e-9 # conversion t to Gt
subset_soc = isric_ocs.multiply(pixel_area).multiply(factor)

#### Geometry Setup 

In [ ]:
# define 1 degree latitude range
latRange_N = ee.Geometry.BBox(-180+0.001, 0, 180-0.001, 90)

#### Single cell 

In [ ]:
out = subset_soc.reduceRegion(reducer=ee.Reducer.sum(), geometry=latRange_N, maxPixels=1e12)
print(out.getInfo())

In [ ]:
fishnet = geemap.fishnet(latRange_N, h_interval=360, v_interval=1, intersect=False)

In [ ]:
# reduce to grid cell
reduced = subset_soc.reduceRegions(collection=fishnet, reducer=ee.Reducer.sum())
# extract and save to df
df = geemap.ee_to_df(reduced)

In [ ]:
df.to_csv('Total_Carbon_per_zone.csv')

#### Analysis 

In [ ]:
# rename for direct label naming in seaborn
df_renamed = df.rename(columns={'south':'latitude', 'sum':'Carbon stocks Gt'})

### Figure

In [ ]:
# Total values
df_renamed.sum(axis=0)

In [ ]:
ax = sns.barplot(data=df_renamed, y='latitude', x='Carbon stocks Gt', orient='h', color='brown', width=1, alpha=0.8)
ax.invert_yaxis()
ax.set_yticks(range(0,90,10))
plt.grid()